In [ ]:
#| default_exp libs.models.unet

## `DoubleConv`

### **Description:**
- Represents a module consisting of two consecutive convolutional layers followed by batch normalization and ReLU activation.

### **Methods:**

 #### `__init__(self, in_channels, out_channels, mid_channels=None)`
   - **Description:**
     - Initializes the DoubleConv module with customizable parameters.
   - **Parameters:**
     - `in_channels` (int): Number of input channels.
     - `out_channels` (int): Number of output channels.
     - `mid_channels` (int, optional): Number of intermediate channels. If not specified, defaults to `out_channels`.
   - **Returns:**
     - None
---

 #### `forward(self, x)`
   - **Description:**
     - Performs forward pass computation through the DoubleConv module.
   - **Parameters:**
     - `x` (torch.Tensor): Input tensor.
   - **Returns:**
     - `torch.Tensor`: Output tensor.
---

## `Down`

### **Description:**
- Represents a downsampling module consisting of max pooling followed by a DoubleConv module.

### **Methods:**

 #### `__init__(self, in_channels, out_channels)`
   - **Description:**
     - Initializes the Down module with customizable parameters.
   - **Parameters:**
     - `in_channels` (int): Number of input channels.
     - `out_channels` (int): Number of output channels.
   - **Returns:**
     - None
---

 #### `forward(self, x)`
   - **Description:**
     - Performs forward pass computation through the Down module.
   - **Parameters:**
     - `x` (torch.Tensor): Input tensor.
   - **Returns:**
     - `torch.Tensor`: Output tensor.
---

## `Up`

### **Description:**
- Represents an upsampling module followed by a DoubleConv module.

### **Methods:**

 #### `__init__(self, in_channels, out_channels, bilinear=True)`
   - **Description:**
     - Initializes the Up module with customizable parameters.
   - **Parameters:**
     - `in_channels` (int): Number of input channels.
     - `out_channels` (int): Number of output channels.
     - `bilinear` (bool): Indicates whether to use bilinear upsampling. Default is True.
   - **Returns:**
     - None
---

 #### `forward(self, x1, x2)`
   - **Description:**
     - Performs forward pass computation through the Up module.
   - **Parameters:**
     - `x1` (torch.Tensor): Input tensor to be upsampled.
     - `x2` (torch.Tensor): Input tensor from the encoder pathway.
   - **Returns:**
     - `torch.Tensor`: Output tensor.
---

## `OutConv`

### **Description:**
- Represents a 1x1 convolutional layer used for the final output.

### **Methods:**

 #### `__init__(self, in_channels, out_channels)`
   - **Description:**
     - Initializes the OutConv module with customizable parameters.
   - **Parameters:**
     - `in_channels` (int): Number of input channels.
     - `out_channels` (int): Number of output channels.
   - **Returns:**
     - None
---

 #### `forward(self, x)`
   - **Description:**
     - Performs forward pass computation through the OutConv module.
   - **Parameters:**
     - `x` (torch.Tensor): Input tensor.
   - **Returns:**
     - `torch.Tensor`: Output tensor.
---

## `UNet`

### **Description:**
- Represents U-Net, an encoder-decoder architecture for semantic segmentation tasks.

### **Methods:**

 #### `__init__(self, n_channels=1, n_classes=6, bilinear=False)`
   - **Description:**
     - Initializes the UNet model with customizable parameters.
   - **Parameters:**
     - `n_channels` (int): Number of input channels. Default is 1.
     - `n_classes` (int): Number of output classes. Default is 6.
     - `bilinear` (bool): Indicates whether to use bilinear interpolation for upsampling. Default is False.
   - **Returns:**
     - None
---

 #### `forward(self, x)`
   - **Description:**
     - Performs forward pass computation through the UNet model.
   - **Parameters:**
     - `x` (torch.Tensor): Input tensor.
   - **Returns:**
     - `torch.Tensor`: Output tensor representing the segmented classes.
---

 #### `use_checkpointing(self)`
   - **Description:**
     - Uses checkpointing to reduce memory consumption during training.
   - **Parameters:**
     - None
   - **Returns:**
     - None
---


In [ ]:
#| export
import torch


class DoubleConv(torch.nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            torch.nn.BatchNorm2d(mid_channels),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(inplace=True)
        )


    def forward(self, x):
        return self.double_conv(x)


class Down(torch.nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = torch.nn.Sequential(
            torch.nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )


    def forward(self, x):
        return self.maxpool_conv(x)


class Up(torch.nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = torch.nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = torch.nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=1)


    def forward(self, x):
        return self.conv(x)


class UNet(torch.nn.Module):
    '''
    U-Net's encoder-decoder architecture
    https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28
    '''

    def __init__(self, n_channels=1, n_classes=6, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))


    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)
